In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
from scripts.load_data import *
from scripts.availabledata import *
from scripts.individual import *
from scripts.cumulative import *
from scripts.bothdatasets import *
from scripts.dataoption import *
from scripts.higher_years import *

import optuna
import numpy as np

In [7]:
configuration = load_configuration("../configuration/configuration.json")

data_individual, data_cumulative, data_student_numbers_first_years, data_student_numbers_higher_years, data_student_numbers_volume, data_latest, data_distances, ensemble_weights = load_data(configuration, StudentYearPrediction.FIRST_YEARS)

dataholder = Cumulative(data_cumulative, data_student_numbers_first_years, configuration)

data_cumulative = dataholder.preprocess()

numerus_fixus_list = configuration["numerus_fixus"]

c:\Users\jjble\Documents\Master\Jaar2_Semester2\Research_Internship\Tijdens\Code\studentprognose\scripts\load_data.py:11: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cumulative = pd.read_csv(paths["path_cumulative"], sep=';', skiprows=[1]) if (paths["path_cumulative"] != "" and os.path.exists(paths["path_cumulative"])) else None
c:\Users\jjble\Documents\Master\Jaar2_Semester2\Research_Internship\Tijdens\Code\studentprognose\scripts\cumulative.py:72: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[key] = data[key].str.replace('.', '')


In [8]:
predict_year = 2023
skip_years = 1
predict_week = 15

In [9]:
predict_week = 15

In [10]:
data_to_predict = data_cumulative[(data_cumulative["Collegejaar"] == predict_year - skip_years) &
                                               (data_cumulative["Weeknummer"] == predict_week)]

dataholder.prepare_data()

full_data = dataholder.data_cumulative.copy()

full_data = full_data[full_data["Collegejaar"] >= 2016]
full_data = full_data[full_data["Collegejaar"] <= predict_year]

data = transform_data(full_data.copy(), 'ts')

In [11]:
def objective(trial):
    total_mae = 0.0
    total_mape = 0.0
    count = 0.0

    learning_rate = trial.suggest_float("learning_rate", 0.0, 1.0)

    for _, row in data_to_predict.iterrows():
        programme = row['Croho groepeernaam']
        herkomst = row["Herkomst"]
        examtype = row["Examentype"]

        # Train/test split
        if programme not in numerus_fixus_list:
            train = data[(data["Collegejaar"] < predict_year - skip_years) & (data["Examentype"] == examtype) & (
                ~data['Croho groepeernaam'].isin(numerus_fixus_list))]
            train_copy = data[(data["Collegejaar"] < predict_year) & (data["Examentype"] == examtype) & (
                ~data['Croho groepeernaam'].isin(numerus_fixus_list))]
        elif programme in numerus_fixus_list:
            train = data[(data["Collegejaar"] < predict_year - skip_years) & (data['Croho groepeernaam'] == programme)]
            train_copy = data[(data["Collegejaar"] < predict_year) & (data['Croho groepeernaam'] == programme)]

        test = data[(data["Collegejaar"] == predict_year - skip_years) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == herkomst)]

        if data_student_numbers_first_years is not None:
            train = train.merge(data_student_numbers_first_years[['Croho groepeernaam', 'Collegejaar', 'Herkomst', 'Aantal_studenten']],
                                on=['Croho groepeernaam', 'Collegejaar', 'Herkomst'])
            train_copy = train_copy.merge(data_student_numbers_first_years[['Croho groepeernaam', 'Collegejaar', 'Herkomst', 'Aantal_studenten']],
                                on=['Croho groepeernaam', 'Collegejaar', 'Herkomst'])

        train = train.drop_duplicates()
        train_copy = train_copy.drop_duplicates()

        train_copy["Collegejaar"] = train_copy["Collegejaar"] - skip_years
        train_copy = train_copy.rename(columns={"Aantal_studenten": "Aantal_studenten_next_year"})

        new_train = train.merge(train_copy[['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst', 'Aantal_studenten_next_year']],
                                on=['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst'], how='left')

        X_train = new_train.drop(['Aantal_studenten_next_year', "Aantal_studenten"], axis=1)
        y_train = new_train.pop('Aantal_studenten_next_year').fillna(0)

        # Encode
        # Specify the numeric and categorical column names
        numeric_cols = ['Collegejaar'] + [str(x) for x in get_weeks_list(38)]
        categorical_cols = ['Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst']

        # Create transformers for numeric and categorical columns
        numeric_transformer = "passthrough"  # No transformation for numeric columns
        categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        # Create the column transformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('numeric', numeric_transformer, numeric_cols),
                ('categorical', categorical_transformer, categorical_cols)
            ])

        # Apply the preprocessing to the training and test data
        X_train = preprocessor.fit_transform(X_train)
        test = preprocessor.transform(test)
        # Model
        model = XGBRegressor(learning_rate=learning_rate)

        model.fit(X_train, y_train)

        predictie = model.predict(test)

        final_prediction = int(round(predictie[0], 0))

        nr_of_students = data_student_numbers_first_years[(data_student_numbers_first_years["Collegejaar"] == predict_year) & (data_student_numbers_first_years["Croho groepeernaam"] == programme) & 
                                                        (data_student_numbers_first_years["Herkomst"] == herkomst)]["Aantal_studenten"]
        
        mae = "???"
        mape = "???"
        if len(nr_of_students) > 0:
            mae = abs(nr_of_students.iloc[0] - final_prediction)
            mape = abs((nr_of_students.iloc[0] - final_prediction) / nr_of_students.iloc[0])

            total_mae += mae
            total_mape += mape
            count += 1.0

        print(f"{programme}, {herkomst}, {final_prediction}, MAE: {mae}, MAPE: {mape}")

    print(f"Final MAE: {total_mae / count}")
    print(f"Final MAPE: {total_mape / count}")

    return np.mean([total_mae / count, total_mape / count])

In [12]:
study = optuna.create_study()
study.optimize(objective, n_trials=20)

print(study.best_params)

[I 2024-05-21 12:02:10,233] A new study created in memory with name: no-name-cfe2d49a-1ddf-47c5-a324-311e66018a87


B Algemene Cultuurwetenschappen, EER, 20, MAE: 6, MAPE: 0.42857142857142855
B Algemene Cultuurwetenschappen, NL, 31, MAE: 2, MAPE: 0.06896551724137931
B Algemene Cultuurwetenschappen, Niet-EER, 10, MAE: 6, MAPE: 1.5
B Artificial Intelligence, EER, 22, MAE: 25, MAPE: 0.5319148936170213
B Artificial Intelligence, NL, 96, MAE: 57, MAPE: 0.37254901960784315
B Artificial Intelligence, Niet-EER, 11, MAE: 4, MAPE: 0.26666666666666666
B Artificial Intelligence, onbekend, 11, MAE: ???, MAPE: ???
B Bedrijfskunde, EER, 19, MAE: 10, MAPE: 0.3448275862068966
B Bedrijfskunde, NL, 454, MAE: 73, MAPE: 0.19160104986876642
B Bedrijfskunde, Niet-EER, 12, MAE: 1, MAPE: 0.09090909090909091
B Bestuurskunde, NL, 80, MAE: 3, MAPE: 0.03614457831325301
B Biology, EER, 75, MAE: 27, MAPE: 0.5625
B Biology, NL, 83, MAE: 7, MAPE: 0.07777777777777778
B Biology, Niet-EER, 2, MAE: 3, MAPE: 0.6
B Biomedische Wetenschappen, EER, 0, MAE: ???, MAPE: ???
B Biomedische Wetenschappen, NL, 100, MAE: 5, MAPE: 0.052631578947368

[I 2024-05-21 12:03:46,783] Trial 0 finished with value: 6.836322257969673 and parameters: {'learning_rate': 0.8179250462770044}. Best is trial 0 with value: 6.836322257969673.


M Theologie & Religiewetenschappen, Niet-EER, 4, MAE: ???, MAPE: ???
Final MAE: 13.011049723756907
Final MAPE: 0.6615947921824391
B Algemene Cultuurwetenschappen, EER, 30, MAE: 16, MAPE: 1.1428571428571428
B Algemene Cultuurwetenschappen, NL, 36, MAE: 7, MAPE: 0.2413793103448276
B Algemene Cultuurwetenschappen, Niet-EER, 3, MAE: 1, MAPE: 0.25
B Artificial Intelligence, EER, 28, MAE: 19, MAPE: 0.40425531914893614
B Artificial Intelligence, NL, 112, MAE: 41, MAPE: 0.2679738562091503
B Artificial Intelligence, Niet-EER, 7, MAE: 8, MAPE: 0.5333333333333333
B Artificial Intelligence, onbekend, 8, MAE: ???, MAPE: ???
B Bedrijfskunde, EER, 36, MAE: 7, MAPE: 0.2413793103448276
B Bedrijfskunde, NL, 429, MAE: 48, MAPE: 0.12598425196850394
B Bedrijfskunde, Niet-EER, 7, MAE: 4, MAPE: 0.36363636363636365
B Bestuurskunde, NL, 82, MAE: 1, MAPE: 0.012048192771084338
B Biology, EER, 77, MAE: 29, MAPE: 0.6041666666666666
B Biology, NL, 83, MAE: 7, MAPE: 0.07777777777777778
B Biology, Niet-EER, 10, MAE: 

[I 2024-05-21 12:05:26,042] Trial 1 finished with value: 5.718717906874679 and parameters: {'learning_rate': 0.39796475684766863}. Best is trial 1 with value: 5.718717906874679.


...

M Theologie & Religiewetenschappen, Niet-EER, 0, MAE: ???, MAPE: ???
Final MAE: 10.98342541436464
Final MAPE: 0.5462098998708217
B Algemene Cultuurwetenschappen, EER, 21, MAE: 7, MAPE: 0.5
B Algemene Cultuurwetenschappen, NL, 31, MAE: 2, MAPE: 0.06896551724137931
B Algemene Cultuurwetenschappen, Niet-EER, 3, MAE: 1, MAPE: 0.25
B Artificial Intelligence, EER, 32, MAE: 15, MAPE: 0.3191489361702128
B Artificial Intelligence, NL, 97, MAE: 56, MAPE: 0.3660130718954248
B Artificial Intelligence, Niet-EER, 8, MAE: 7, MAPE: 0.4666666666666667
B Artificial Intelligence, onbekend, 6, MAE: ???, MAPE: ???
B Bedrijfskunde, EER, 43, MAE: 14, MAPE: 0.4827586206896552
B Bedrijfskunde, NL, 425, MAE: 44, MAPE: 0.11548556430446194
B Bedrijfskunde, Niet-EER, 8, MAE: 3, MAPE: 0.2727272727272727
B Bestuurskunde, NL, 90, MAE: 7, MAPE: 0.08433734939759036
B Biology, EER, 67, MAE: 19, MAPE: 0.3958333333333333
B Biology, NL, 74, MAE: 16, MAPE: 0.17777777777777778
B Biology, Niet-EER, 10, MAE: 5, MAPE: 1.0
B Bio

[I 2024-05-21 12:36:25,141] Trial 18 finished with value: 5.345696570843829 and parameters: {'learning_rate': 0.10503855119228139}. Best is trial 3 with value: 5.332997786480856.


M Theologie & Religiewetenschappen, Niet-EER, 2, MAE: ???, MAPE: ???
Final MAE: 10.237569060773481
Final MAPE: 0.45382408091417814
B Algemene Cultuurwetenschappen, EER, 21, MAE: 7, MAPE: 0.5
B Algemene Cultuurwetenschappen, NL, 26, MAE: 3, MAPE: 0.10344827586206896
B Algemene Cultuurwetenschappen, Niet-EER, 2, MAE: 2, MAPE: 0.5
B Artificial Intelligence, EER, 31, MAE: 16, MAPE: 0.3404255319148936
B Artificial Intelligence, NL, 118, MAE: 35, MAPE: 0.22875816993464052
B Artificial Intelligence, Niet-EER, 7, MAE: 8, MAPE: 0.5333333333333333
B Artificial Intelligence, onbekend, 9, MAE: ???, MAPE: ???
B Bedrijfskunde, EER, 32, MAE: 3, MAPE: 0.10344827586206896
B Bedrijfskunde, NL, 418, MAE: 37, MAPE: 0.09711286089238845
B Bedrijfskunde, Niet-EER, 8, MAE: 3, MAPE: 0.2727272727272727
B Bestuurskunde, NL, 89, MAE: 6, MAPE: 0.07228915662650602
B Biology, EER, 74, MAE: 26, MAPE: 0.5416666666666666
B Biology, NL, 82, MAE: 8, MAPE: 0.08888888888888889
B Biology, Niet-EER, 10, MAE: 5, MAPE: 1.0
B B

[I 2024-05-21 12:37:57,825] Trial 19 finished with value: 5.5007926176504105 and parameters: {'learning_rate': 0.6882360970588184}. Best is trial 3 with value: 5.332997786480856.


M Theologie & Religiewetenschappen, Niet-EER, 0, MAE: ???, MAPE: ???
Final MAE: 10.370165745856353
Final MAPE: 0.631419489444469
{'learning_rate': 0.28095298292393744}
